In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [112]:
train_data_path = '../data/train.csv'
test_data_path = '../data/test.csv'

train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

# print(train_data.info())
# print(test_data.info())


In [113]:
# 1. 给test data的survived 补零，和train data合并，用于特征工程处理
test_data['Survived'] = 0
train_test = train_data.append(test_data)
train_test.shape
# train_test.info()
# test_data.shape
# train_data.shape



D:\ProgramFiles\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


(1309, 12)

In [114]:
# 2. ‘Embarked’：判断是否为空，否则以众数(mode().iloc[0])进行填充
if train_test['Embarked'].isnull().sum() != 0:
    train_test['Embarked'].fillna(train_test['Embarked'].mode().iloc[0], inplace=True)

# 对 ‘Embarked’这一列进行拆分，类似于onehot编码(get_dummies())    
emb = pd.get_dummies(train_test['Embarked'], prefix='Embarked')
train_test = pd.concat([train_test, emb], axis=1)



In [115]:
# 3. 'Sex'：无缺失值，直接onehot
sex = pd.get_dummies(train_test['Sex'], prefix='Sex')
train_test = pd.concat([train_test, sex], axis=1)


In [116]:
# 4. 'Name'：从名字中提取称呼和姓名，
name = train_test['Name'].str.extract(",([\S\s]{1,})\.([\S\s]{1,})[\(]{0,1}")
title = name[0]
name = name[1]
 
train_test['Title'] = title
train_test['RealName'] = name
    

In [117]:
# 5. 将 称呼 进行分列
title = pd.get_dummies(train_test['Title'], prefix='Title')
train_test = pd.concat([train_test, title], axis=1)



In [118]:
# 6. ‘Fare’：填充缺失值，按Pclass等级的均价来填
if train_test['Fare'].isnull().sum() != 0:
    train_test['Fare'] = train_test[['Fare']].fillna(train_test.groupby('Pclass').transform('mean'))

# 泰坦尼克号中有家庭团体票（分析Ticket号可以得到），所以需要将团体票分到每个人
group_ticket = train_test['Fare'].groupby(by=train_test['Ticket']).transform('count')
train_test['Fare'] = train_test['Fare'] / group_ticket
    

In [119]:
# 7. 给票价分级，再分列
def fare_category(fare):
        if fare <= 4:
            return 0
        elif fare <= 10:
            return 1
        elif fare <= 30:
            return 2
        elif fare <= 45:
            return 3
        else:
            return 4
train_test['Fare_Category'] = train_test['Fare'].map(fare_category)

fare_cat_dummies_df = pd.get_dummies(train_test['Fare_Category'],prefix=train_test[['Fare_Category']].columns[0])
train_test = pd.concat([train_test, fare_cat_dummies_df], axis=1)
train_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 44 columns):
Age                                1046 non-null float64
Cabin                              295 non-null object
Embarked                           1309 non-null object
Fare                               1309 non-null float64
Name                               1309 non-null object
Parch                              1309 non-null int64
PassengerId                        1309 non-null int64
Pclass                             1309 non-null int64
Sex                                1309 non-null object
SibSp                              1309 non-null int64
Survived                           1309 non-null int64
Ticket                             1309 non-null object
Embarked_C                         1309 non-null uint8
Embarked_Q                         1309 non-null uint8
Embarked_S                         1309 non-null uint8
Sex_female                         1309 non-null uint8
Sex_ma

In [124]:
train_test[['Fare_Category','Fare']].head(10)
p1_mean = train_test['Fare'].groupby(by=train_test['Pclass']).mean().get([1]).values[0]
p2_mean = train_test['Fare'].groupby(by=train_test['Pclass']).mean().get([2]).values[0]
p3_mean = train_test['Fare'].groupby(by=train_test['Pclass']).mean().get([3]).values[0]


33.91050030959751